In [21]:
import pandas as pd
import sqlite3
import pickle
import gc

from functools import lru_cache

# Calculate reference MidPrices for random moments

In [13]:
TRADES_DIR = '../TRADES/USD_TOM_trades_Micex_02_10.feather'
LOB_DIR = '../Data_lob/'

RANDOM_MOMENTS_ALIGNED_OUTPUT = '../0_CommonFiles/random_moments_aligned_to_1000_2000.pkl'
MIDPRICE_DB_PATH = '../0_CommonFiles/mid.db3'

In [24]:
class LOBCache:
    def __init__(self):
        pass
    
    def get_lob(self, datetime):
        filename = datetime.strftime('LOB_%m%d.feather')
        return self.__get_lob_by_filename(filename)
        pass
    
    @lru_cache(maxsize=1)
    def __get_lob_by_filename(self, filename):
        print(filename)
        lob = pd.read_feather(LOB_DIR + filename).dropna()
        lob['BID_SIZE10'] = 99_999_999
        lob['ASK_SIZE10'] = 99_999_999
        return lob

In [25]:
cache = LOBCache()

In [12]:
with open(RANDOM_MOMENTS_ALIGNED_OUTPUT, 'rb') as file:
    moments = pickle.load(file)
len(moments)

2712

In [32]:
#CREATE TABLE MIDPRICE(MOMENT TEXT, PRICE REAL, PRIMARY KEY(MOMENT));
mid_sql = sqlite3.connect(MIDPRICE_DB_PATH)
mid_sql.execute('CREATE TABLE IF NOT EXISTS MIDPRICE(MOMENT TEXT, PRICE REAL, PRIMARY KEY(MOMENT));')
cur=mid_sql.cursor()

In [33]:
for moment in moments:
    record = (str(moment),)
    query='select exists(select 1 from MIDPRICE where MOMENT=? collate nocase) limit 1'
    # 'query' RETURNS 1 IF USERNAME EXISTS OR 0 IF NOT, AS INTEGER(MAYBE). 'collate nocase'= CASE INSENSITIVE, IT'S OPTIONAL
    check=cur.execute(query, record) 
    if check.fetchone()[0]==1:
        print(f'Moment already available ' + str(record))
        continue

    query='INSERT OR REPLACE INTO MIDPRICE(MOMENT, PRICE) VALUES(?, ?);'
    
    lob = cache.get_lob(moment)
    index = lob.Time.searchsorted(moment, side = 'right')
    if len(lob) > 0:
        row = lob.iloc[index]
        mid = (row['BID_PRICE1'] + row['ASK_PRICE1']) / 2
        mid = round(mid, 5)
        record = (str(moment), mid)
    else:
        record = (str(moment), -1.0)
    
    print(record)
    cur.execute(query, record)
    gc.collect()

LOB_0201.feather
('2021-02-01 18:10:15.611222', 76.08875)
('2021-02-01 22:20:17.074682', 75.9925)
('2021-02-01 23:31:08.196003', 76.13)
('2021-02-01 13:23:14.931749', 75.6)
('2021-02-01 13:31:09.506345', 75.6275)
('2021-02-01 11:23:02.279950', 75.4025)
('2021-02-01 12:12:26.394156', 75.60875)
('2021-02-01 12:23:23.321660', 75.45625)
('2021-02-01 12:34:16.916168', 75.37875)
('2021-02-01 16:31:21.180792', 75.945)
('2021-02-01 17:10:51.083060', 76.015)
('2021-02-01 17:33:43.900099', 76.00625)
LOB_0202.feather
('2021-02-02 19:10:04.621620', 75.55)
('2021-02-02 23:34:13.063881', 76.1925)
('2021-02-02 14:19:32.303616', 75.82625)
('2021-02-02 12:08:21.873407', 75.79875)
('2021-02-02 12:33:22.669540', 75.755)
('2021-02-02 13:08:56.870705', 75.85875)
('2021-02-02 13:20:08.272736', 75.95875)
('2021-02-02 13:32:27.706510', 75.97625)
('2021-02-02 10:12:20.698970', 75.8675)
('2021-02-02 10:23:21.168553', 76.0525)
('2021-02-02 10:33:01.549783', 75.975)
('2021-02-02 20:23:43.402557', 75.9)
('2021-02-

('2021-02-18 13:33:43.513027', 73.7125)
('2021-02-18 12:12:30.279544', 73.6625)
('2021-02-18 12:21:24.463454', 73.66875)
('2021-02-18 12:30:22.589689', 73.73875)
('2021-02-18 19:22:44.822327', 73.93375)
('2021-02-18 17:10:02.874750', 73.79875)
('2021-02-18 17:22:30.320591', 73.85375)
('2021-02-18 17:33:07.700196', 73.83375)
('2021-02-18 22:21:42.523512', 73.91625)
('2021-02-18 15:09:16.017410', 73.74875)
('2021-02-18 15:30:31.427706', 73.69)
LOB_0219.feather
('2021-02-19 13:10:39.207253', 73.95125)
('2021-02-19 13:19:18.869391', 73.885)
('2021-02-19 13:33:20.978034', 73.935)
('2021-02-19 11:23:20.373008', 74.02875)
('2021-02-19 11:34:05.444691', 74.0125)
('2021-02-19 19:23:02.616986', 73.94625)
('2021-02-19 19:31:13.484347', 73.89625)
('2021-02-19 15:09:31.508652', 73.9525)
('2021-02-19 15:23:15.566494', 73.90375)
('2021-02-19 15:30:01.056083', 73.86625)
('2021-02-19 20:11:39.280393', 74.0025)
('2021-02-19 20:19:00.444883', 73.975)
('2021-02-19 18:11:35.975900', 74.01875)
('2021-02-19 

('2021-03-05 12:31:12.222342', 74.56125)
('2021-03-05 17:10:50.864565', 74.23125)
('2021-03-05 17:23:36.766029', 74.22375)
('2021-03-05 17:34:34.011424', 74.2475)
('2021-03-05 11:10:57.239586', 74.465)
('2021-03-05 11:22:45.446631', 74.5175)
('2021-03-05 11:32:48.658937', 74.46125)
('2021-03-05 23:23:16.005335', 74.20875)
('2021-03-05 23:34:51.717733', 74.23625)
('2021-03-05 15:12:07.812169', 74.54125)
('2021-03-05 15:30:31.348684', 74.49375)
('2021-03-05 18:11:33.937137', 74.25625)
LOB_0309.feather
('2021-03-09 11:23:17.103537', 74.19125)
('2021-03-09 11:30:58.866724', 74.10125)
('2021-03-09 13:11:41.950483', 73.95375)
('2021-03-09 13:19:30.404135', 74.02625)
('2021-03-09 15:12:04.751246', 73.94875)
('2021-03-09 15:19:26.404390', 73.90625)
('2021-03-09 12:10:27.790058', 74.0925)
('2021-03-09 12:22:51.640772', 74.1075)
('2021-03-09 12:34:32.663267', 74.11625)
('2021-03-09 22:12:29.392791', 73.90125)
('2021-03-09 22:34:49.323114', 73.925)
('2021-03-09 17:11:24.577624', 74.01625)
('2021-

('2021-03-23 19:20:58.819245', 76.0325)
('2021-03-23 19:34:30.143823', 76.10875)
('2021-03-23 16:10:25.065455', 75.9925)
('2021-03-23 16:20:11.870625', 75.93125)
('2021-03-23 16:32:20.867374', 75.85875)
('2021-03-23 21:19:26.765577', 76.2225)
('2021-03-23 21:32:30.198631', 76.23125)
('2021-03-23 18:12:33.334813', 76.04125)
('2021-03-23 18:23:04.107478', 76.03625)
('2021-03-23 13:12:23.441914', 75.9675)
('2021-03-23 13:22:58.405099', 75.935)
('2021-03-23 13:33:33.457131', 75.90375)
('2021-03-23 12:11:56.915162', 75.695)
('2021-03-23 12:20:34.105840', 75.87125)
('2021-03-23 12:33:45.485362', 75.88875)
LOB_0324.feather
('2021-03-24 15:11:10.922041', 76.115)
('2021-03-24 15:19:24.806422', 76.155)
('2021-03-24 15:30:44.530249', 76.10125)
('2021-03-24 19:11:34.631239', 76.365)
('2021-03-24 19:22:11.425421', 76.33875)
('2021-03-24 19:33:44.517166', 76.3325)
('2021-03-24 21:21:09.013533', 76.2175)
('2021-03-24 16:11:14.137692', 76.2975)
('2021-03-24 16:19:15.330097', 76.22625)
('2021-03-24 16:

('2021-04-12 20:10:41.276140', 77.34)
('2021-04-12 20:34:07.929708', 77.315)
('2021-04-12 19:19:42.629245', 77.38)
('2021-04-12 11:23:35.434443', 77.34125)
('2021-04-12 11:34:04.985695', 77.30375)
('2021-04-12 15:11:29.046598', 77.21125)
('2021-04-12 15:20:12.999478', 77.25375)
('2021-04-12 13:11:52.478208', 77.3125)
('2021-04-12 13:21:35.008510', 77.22)
('2021-04-12 13:30:27.157951', 77.19875)
('2021-04-12 10:10:05.589132', 77.61625)
('2021-04-12 10:31:31.536276', 77.68875)
('2021-04-12 18:19:20.057854', 77.40125)
LOB_0413.feather
('2021-04-13 14:21:41.787069', 77.20125)
('2021-04-13 14:34:07.700521', 77.1975)
('2021-04-13 22:19:23.215738', 75.90125)
('2021-04-13 10:11:01.374341', 77.505)
('2021-04-13 10:22:08.304432', 77.395)
('2021-04-13 10:34:53.130842', 77.12125)
('2021-04-13 19:09:38.280487', 76.2425)
('2021-04-13 15:08:20.197520', 77.26625)
('2021-04-13 15:34:20.544887', 77.16875)
('2021-04-13 11:07:41.450644', 77.2)
('2021-04-13 11:20:22.903690', 77.0425)
('2021-04-13 11:33:48.

...

('2021-07-13 19:23:05.257732', 74.0525)
LOB_0714.feather
('2021-07-14 15:11:49.995027', 74.06625)
('2021-07-14 15:19:29.314822', 74.06375)
('2021-07-14 15:33:51.282468', 73.98625)
('2021-07-14 18:32:38.801787', 74.18875)
('2021-07-14 14:11:05.567267', 74.18125)
('2021-07-14 14:32:06.063921', 74.0525)
('2021-07-14 11:23:38.475056', 74.11875)
('2021-07-14 16:12:07.732424', 74.03375)
('2021-07-14 16:23:15.952980', 74.04375)
('2021-07-14 16:31:25.680316', 74.055)
('2021-07-14 12:10:54.079838', 74.15625)
('2021-07-14 12:23:15.562731', 74.1)
('2021-07-14 12:30:45.026087', 74.12125)
LOB_0715.feather
('2021-07-15 13:11:54.830685', 74.13375)
('2021-07-15 13:20:56.831678', 74.14375)
('2021-07-15 16:23:13.283252', 74.15625)
('2021-07-15 16:33:25.953771', 74.0975)
('2021-07-15 17:32:34.339478', 74.13)
('2021-07-15 15:10:12.324605', 74.12)
('2021-07-15 15:22:44.596861', 74.10625)
('2021-07-15 15:32:27.757427', 74.09625)
('2021-07-15 22:09:01.515463', 74.37875)
('2021-07-15 22:30:20.062422', 74.3675

('2021-07-30 14:32:44.433355', 73.0)
('2021-07-30 16:11:09.902798', 72.94)
('2021-07-30 16:21:21.780194', 72.975)
('2021-07-30 17:22:48.366309', 72.915)
('2021-07-30 17:31:17.646566', 72.9575)
LOB_0802.feather
('2021-08-02 16:12:11.543378', 72.84375)
('2021-08-02 16:21:15.627076', 72.825)
('2021-08-02 16:34:42.284807', 72.77625)
('2021-08-02 11:10:59.748678', 72.91875)
('2021-08-02 11:20:55.996381', 72.93875)
('2021-08-02 11:32:19.270165', 72.90125)
('2021-08-02 15:11:04.340926', 72.74125)
('2021-08-02 15:22:44.388145', 72.75125)
('2021-08-02 15:32:05.692890', 72.75625)
('2021-08-02 13:10:56.953593', 72.91875)
('2021-08-02 13:22:48.386837', 72.89)
('2021-08-02 13:32:41.682725', 72.8875)
('2021-08-02 14:12:07.751819', 72.88625)
('2021-08-02 14:22:07.952640', 72.865)
('2021-08-02 14:32:59.795668', 72.86875)
('2021-08-02 17:11:46.890943', 72.81625)
('2021-08-02 17:22:50.094788', 72.81125)
LOB_0803.feather
('2021-08-03 10:11:25.761841', 72.95375)
('2021-08-03 10:23:21.168859', 72.88875)
('

('2021-08-18 11:34:29.449110', 73.49625)
LOB_0819.feather
('2021-08-19 16:11:29.176136', 74.145)
('2021-08-19 16:22:48.053935', 74.225)
('2021-08-19 16:33:26.572214', 74.2375)
('2021-08-19 15:11:14.811748', 74.13)
('2021-08-19 15:22:39.704230', 74.10375)
('2021-08-19 15:30:55.137026', 74.12)
('2021-08-19 14:08:31.849331', 74.14875)
('2021-08-19 14:23:09.038660', 74.18875)
('2021-08-19 14:32:28.596146', 74.20625)
('2021-08-19 21:31:00.274240', 74.34875)
('2021-08-19 18:31:45.342386', 74.30125)
('2021-08-19 19:08:31.023958', 74.275)
('2021-08-19 13:08:52.269039', 74.105)
('2021-08-19 13:32:28.137779', 74.14625)
('2021-08-19 11:08:12.987747', 74.20125)
('2021-08-19 11:20:48.693971', 74.1975)
('2021-08-19 11:34:24.605505', 74.08625)
LOB_0820.feather
('2021-08-20 19:08:51.127726', 74.35125)
('2021-08-20 19:21:00.603833', 74.3225)
('2021-08-20 19:30:20.099779', 74.30375)
('2021-08-20 21:09:38.392682', 74.30125)
('2021-08-20 21:33:25.126018', 74.31625)
('2021-08-20 13:10:57.195106', 74.56625)

('2021-09-08 13:10:07.815956', 73.39625)
('2021-09-08 13:18:54.355119', 73.3775)
('2021-09-08 13:31:23.460932', 73.31)
('2021-09-08 15:10:24.784590', 73.2675)
LOB_0909.feather
('2021-09-09 16:19:14.658187', 73.04875)
('2021-09-09 19:19:38.684098', 72.86375)
('2021-09-09 14:33:10.900937', 72.9675)
('2021-09-09 10:23:11.307976', 73.1525)
('2021-09-09 10:30:00.042225', 73.16875)
('2021-09-09 21:22:19.628040', 72.98875)
('2021-09-09 15:10:03.475517', 72.88)
('2021-09-09 15:31:32.384014', 73.055)
LOB_0910.feather
('2021-09-10 14:23:07.836875', 72.74375)
('2021-09-10 19:31:52.965988', 73.0775)
('2021-09-10 18:11:08.339507', 73.13625)
('2021-09-10 18:23:27.002815', 73.11375)
('2021-09-10 12:11:23.939611', 72.76375)
('2021-09-10 12:23:42.241008', 72.77)
('2021-09-10 12:34:25.223093', 72.77125)
('2021-09-10 10:11:34.200083', 72.7875)
('2021-09-10 10:23:33.357151', 72.7825)
('2021-09-10 10:31:42.699524', 72.75625)
('2021-09-10 22:09:09.971187', 73.16875)
('2021-09-10 13:19:05.420535', 72.75625)


('2021-09-29 15:20:01.175539', 72.65)
('2021-09-29 15:34:01.996527', 72.65375)
('2021-09-29 14:19:57.043250', 72.7875)
('2021-09-29 14:30:19.348360', 72.79)
('2021-09-29 17:11:31.586085', 72.835)
('2021-09-29 12:08:34.099295', 72.75)
('2021-09-29 12:22:48.013732', 72.76625)
('2021-09-29 12:32:20.297246', 72.76125)
('2021-09-29 21:09:08.867589', 72.94)
LOB_0930.feather
('2021-09-30 18:12:09.406097', 72.675)
('2021-09-30 18:34:54.676050', 72.65)
('2021-09-30 12:12:15.034739', 72.66125)
('2021-09-30 12:23:36.461834', 72.6825)
('2021-09-30 12:32:34.282676', 72.69375)
('2021-09-30 13:21:42.322816', 72.76)
('2021-09-30 13:32:49.016717', 72.86)
('2021-09-30 20:09:28.006496', 72.71375)
('2021-09-30 20:32:30.585416', 72.8025)
('2021-09-30 11:08:00.570398', 72.5975)
('2021-09-30 11:22:42.345436', 72.62125)
('2021-09-30 17:12:33.890044', 72.81875)
('2021-09-30 17:22:52.258829', 72.85375)
('2021-09-30 17:32:26.537043', 72.78625)
('2021-09-30 14:11:46.240916', 72.86125)
('2021-09-30 10:11:06.690525

In [34]:
mid_sql.commit()

In [35]:
cur.close()
mid_sql.close()